# A training log

- `step 1000`: train loss 5.9037, `val loss 5.8993`

![At 1000 epochs](./training-log/1000-epochs.png "1000 Epochs")

- `step 1500`: train loss 5.6095, `val loss 5.5964`
- interestingly, more gibberish. will run more epochs before checking again.

![At 1500 epochs](./training-log/1500-epochs.png "1500 Epochs")

- `step 3000`: train loss 5.0643, `val loss 5.0579`
- seems very "news"-y?

![At 3000 epochs](./training-log/3000-epochs.png "3000 Epochs")

- `step 5500`: train loss 4.6092, `val loss 4.6042`
- output is looking more coherent, but still very "news"-y
- let's wait until 10k epochs and then see if we need to bump up the learning rate

![At 5500 epochs](./training-log/5500-epochs.png "5500 Epochs")

- `step 50k`, `val loss: 3.628270149230957`
- ramped up training to 600k overall steps and left overnight
- brought in cosine annealing with linear warmup on the first 2k steps (per paper)
- implemented checkpoints so I can resume training across instances
- switched to the A10G GPU instances with 4 cores
- pulled entire training/val datasets into memory which greatly increased GPU throughput (see images below):

![Train to Val before Data in Mem](./training-log/train-to-val-before.png "Train to Val before data in mem")

Before data pulled into memory. Left is GPU utilization seeking across the 18GB `train.bin` file, right is from 85MB `val.bin` file. You can see the huge discrepancy in throughput randomly reading from each via `np.memmap`.

![Train to Val after Data in Mem](./training-log/train-to-val-after.png "Train to Val after data in mem")

Utilization results after pulling the whole dataset into memory. Notes from left-to-right with dips between (representing code execution moving to a new stage):

1. GPU Utilization of pulling from `train.bin` for loss estimation - much higher without the dips going to 0% utilization.
2. GPU Utilization of pulling from `val.bin` for loss estimation - notice that this is the same as randomly pulling from `train.bin`.
3. GPU Utilization of randomly pulling from `train.bin` for training. Somehow this is even higher than randomly pulling from the file for loss estimation with dips sometimes not dropping all the way to 50%. I'm guessing this is because we are also saving to the checkpoint file vs. not in the training case.

![At 50k epochs](./training-log/50k-epochs.png "50k Epochs")

After training for some time, you can see much better results with a loss that's in the `~3.6` range. The output seems a lot more conversational. I'm going to keep this running (likely for a full day or 2) until I get diminishing returns on loss. I would love to see if I could get away with even faster using an A100 or H100, but I don't want to spend that kind of cash on idle given how noobish I still am at all of this stuff and because I'd have to either use a 3rd party like Lambda (which is more of a pain) and/or upgrade to the next pricing tier.

- `step 73k-93k`: seems like our loss has stalled a bit. Not sure why. Thinking.

![Stalled loss](./training-log/73k-93k.png "Stalled loss")